In [43]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = ' # Change it for your AWS endpoint
USER = 'postgres'
#PASSWORD = str(input('Database Password: '))
PASSWORD = ''
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")

In [44]:
df = pd.read_sql('products', engine)

(df.head())

,id,product_name,category,product_description,price,location,page_id,create_time
0,ac2140ae-f0d5-4fe7-ac08-df0f109fd734,"Second-Hand Sofas, Couches & Armchairs for Sal...",N/A,N/A,N/A,N/A,1426592234,2022-02-26
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",1426704584,2022-02-26
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",1426704579,2022-02-26
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",1426704576,2022-02-26
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",1426704575,2022-02-26


In [45]:
df.dtypes

id                             object
product_name                   object
category                       object
product_description            object
price                          object
location                       object
page_id                         int64
create_time            datetime64[ns]
dtype: object

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8091 entries, 0 to 8090
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   8091 non-null   object        
 1   product_name         8091 non-null   object        
 2   category             8091 non-null   object        
 3   product_description  8091 non-null   object        
 4   price                8091 non-null   object        
 5   location             8091 non-null   object        
 6   page_id              8091 non-null   int64         
 7   create_time          8091 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 505.8+ KB


In [48]:
questionable_columns = ['id','product_name','price', 'category','product_description', 'location', 'page_id','create_time']
na_counts = {col: 0 for col in questionable_columns}
for col in questionable_columns:
    na_counts[col] = df[col][df[col] == 'N/A'].count() #identify N/A 
    
print(na_counts)
print(df.shape)

{'id': 0, 'product_name': 0, 'price': 935, 'category': 935, 'product_description': 935, 'location': 935, 'page_id': 0, 'create_time': 0}
(8091, 8)


In [49]:
data = df.loc[df["price"] != 'N/A'] #removes all rows with 'N/A' for prices
data.shape

(7156, 8)

In [51]:
data['price'] = data['price'].str.strip('£') #removes £ 
data["price"] = [float(str(i).replace(",", "")) for i in data["price"]] 

/tmp/ipykernel_7281/2010094058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['price'].str.strip('£')
/tmp/ipykernel_7281/2010094058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["price"] = [float(str(i).replace(",", "")) for i in data["price"]]


In [53]:
data['price'] = data['price'].astype('float64')

/tmp/ipykernel_7281/1114909614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['price'].astype('float64')


In [62]:
data.dtypes

id                             object
product_name                   object
category                       object
product_description            object
price                         float64
location                       object
page_id                         int64
create_time            datetime64[ns]
category0                      object
category1                      object
category2                      object
category3                      object
category4                      object
dtype: object

In [55]:
# as there are lots of potential categories for 'category', if we split it into different columns, we can have fewer number of categories as they won't be seperated as much
col = 'category'
no_cols = 5
character = '/'
data[[col+str(i) for i in range(no_cols)]] = (data[col].str.split(character, expand=True))
df2 = data.drop(col, axis=1) # drops empty column
for i in range(no_cols):
            if i > 2:
                df3 = df2.drop(col+str(i), axis=1)

/tmp/ipykernel_7281/2462911171.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[[col+str(i) for i in range(no_cols)]] = (data[col].str.split(character, expand=True))
/tmp/ipykernel_7281/2462911171.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[[col+str(i) for i in range(no_cols)]] = (data[col].str.split(character, expand=True))
/tmp/ipykernel_7281/2462911171.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [57]:
df3.head()

,id,product_name,product_description,price,location,page_id,create_time,category0,category1,category2,category3
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Mirror wall art. Posted by Nisha in Dining, Li...",5.0,"Wokingham, Berkshire",1426704584,2022-02-26,Home & Garden,"Dining, Living Room Furniture","Mirrors, Clocks & Ornaments",None
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Morphy Richard’s (model no 48755)Stainless ste...,20.0,"Inverness, Highland",1426704579,2022-02-26,Home & Garden,Other Household Goods,None,None
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",I have 2 of these - collection only as I don’t...,20.0,"Skegness, Lincolnshire",1426704576,2022-02-26,Home & Garden,Garden & Patio,Outdoor Settings & Furniture,None
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,Great reclaimed army ammunition box used as co...,115.0,"Radstock, Somerset",1426704575,2022-02-26,Home & Garden,"Dining, Living Room Furniture",Other,None
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,New Design Shannon Corner sofa 5 Seater Avail...,450.0,"Delph, Manchester",1426704570,2022-02-26,Home & Garden,"Dining, Living Room Furniture","Sofas, Armchairs & Suites",Private


In [58]:
df3.dtypes

id                             object
product_name                   object
product_description            object
price                         float64
location                       object
page_id                         int64
create_time            datetime64[ns]
category0                      object
category1                      object
category2                      object
category3                      object
dtype: object

In [63]:
df3['category0'] = df3['category0'].astype('category') 
df3['category1'] = df3['category1'].astype('category') 
df3['category2'] = df3['category2'].astype('category') 
df3['category3'] = df3['category3'].astype('category') 

In [72]:
df3['category1'].describe()

count                                7156
unique                                115
top        Computers, Laptops & Netbooks 
freq                                  604
Name: category1, dtype: object

In [74]:
df3.dtypes

id                             object
product_name                   object
product_description            object
price                         float64
location                       object
page_id                         int64
create_time            datetime64[ns]
category0                    category
category1                    category
category2                    category
category3                    category
dtype: object

In [73]:
df3.to_csv('clean_data.csv', index=False)